In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from collections import defaultdict
import pickle
import torch
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
import torchvision
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC

In [3]:
def load_data(path, label):
    with open(path, 'rb') as iffile:
        features = pickle.load(iffile)
    layer = 'flatten'
    features = features[layer]
    features_matrix = None

    for i in range(len(features)):
        if isinstance(features_matrix, torch.Tensor):
            features_matrix = torch.cat((   features_matrix, features[i]))
        else:
            features_matrix = features[i]

    features = None    
    features_matrix = torch.flatten(features_matrix, start_dim=1, end_dim=-1)
    y = np.empty(features_matrix.shape[0])
    y.fill(label)
    return [features_matrix, y]


In [4]:
def unison_shuffled_copies(x_ben, y_ben, x_adv, y_adv):
    X = np.concatenate((x_ben, x_adv), axis=0)
    y = np.concatenate((y_ben, y_adv), axis=0)
    assert len(X) == len(y)
    p = np.random.permutation(len(X))
    return X[p], y[p]

In [5]:
base_path = "/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666"
def get_paths(base_path, attack_type):
    relu_dir = "ReLUs"
    train_benign = f"ReLUs_CW_train_benign_5120_integrated-False.pkl"
    train_adversarial = f"ReLUs_CW_train_adversarial_5120_integrated-False.pkl"
    test_benign = f"ReLUs_CW_test_benign_5120_integrated-False.pkl"
    test_adversarial = f"ReLUs_CW_test_adversarial_5120_integrated-False.pkl"

    train_benign = os.path.join(base_path, relu_dir, train_benign)
    train_adversarial = os.path.join(base_path, relu_dir,  train_adversarial)
    test_benign = os.path.join(base_path,  relu_dir, test_benign)
    test_adversarial = os.path.join(base_path,  relu_dir, test_adversarial)

    all_paths = [train_benign,train_adversarial,test_benign,test_adversarial]

    return all_paths


In [6]:
attack_type="CW"
train_benign,train_adversarial,test_benign,test_adversarial = get_paths(base_path, attack_type)

In [7]:
train_benign

'/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/ReLUs/ReLUs_CW_train_benign_5120_integrated-False.pkl'

In [8]:
train_benign = load_data(train_benign, 1)
train_adversarial = load_data(train_adversarial, -1)
test_benign = load_data(test_benign, 1)
test_adversarial = load_data(test_adversarial, -1)

In [9]:
min_train = min(train_benign[0].shape[0], train_adversarial[0].shape[0])
min_test = min(test_benign[0].shape[0], test_adversarial[0].shape[0])

In [10]:
# Subsample the larger sets

rand_indices = np.random.choice(train_benign[0].shape[0], size=min_train, replace=False)
train_benign[0] = train_benign[0][rand_indices] 
train_benign[1] = train_benign[1][rand_indices]

rand_indices = np.random.choice(test_benign[0].shape[0], size=min_test, replace=False)
test_benign[0] = test_benign[0][rand_indices] 
test_benign[1] = test_benign[1][rand_indices]


In [11]:
X_train, y_train = unison_shuffled_copies(train_benign[0], train_benign[1], train_adversarial[0], train_adversarial[1])

X_test, y_test = unison_shuffled_copies(test_benign[0], test_benign[1], test_adversarial[0], test_adversarial[1])


In [16]:


clf = SVC(C=0.7, gamma=0.075)
clf.fit(X_train, y_train)


SVC(C=0.7, gamma=0.075)

In [17]:

# Testing
x_train_pred = clf.predict(X_train)
train_acc = len(np.where(x_train_pred == y_train)[0]) / len(X_train)

x_test_pred = clf.predict(X_test)
test_acc = len(np.where(x_test_pred == y_test)[0]) / len(X_test)




In [18]:
print("Training acc : ", train_acc)
print("Test acc : ", test_acc)


Training acc :  0.94677734375
Test acc :  0.90869140625


In [19]:
print("Training acc : ", train_acc)
print("Test acc : ", test_acc)


Training acc :  0.94677734375
Test acc :  0.90869140625


In [72]:

# Test classwise
x_train_ben_pred = clf.predict(train_benign[0])
train_ben_acc = len(np.where(x_train_ben_pred == train_benign[1])[0]) / len(x_train_ben_pred)

x_train_adv_pred = clf.predict(train_adversarial[0])
train_adv_acc = len(np.where(x_train_adv_pred == train_adversarial[1])[0]) / len(x_train_adv_pred)




x_test_ben_pred = clf.predict(test_benign[0])
test_ben_acc = len(np.where(x_test_ben_pred == test_benign[1])[0]) / len(x_test_ben_pred)


x_test_adv_pred = clf.predict(test_adversarial[0])
test_adv_acc = len(np.where(x_test_adv_pred == test_adversarial[1])[0]) / len(x_test_adv_pred)

In [73]:

print("Training benign acc : ", train_ben_acc)
print("Training adversarial acc : ", train_adv_acc)
print("-------------------------------------")
print("Testing benign acc : ", test_ben_acc)
print("Testing adversarial acc : ", test_adv_acc)


Training benign acc :  0.963671875
Training adversarial acc :  0.925390625
-------------------------------------
Testing benign acc :  0.8892578125
Testing adversarial acc :  0.9181640625


# Extracting support vectors and associated parameters

In [12]:
# Manual svm implementation

X = torch.from_numpy(X_test)
# Features to extract
SV = clf.support_vectors_
IC = clf.intercept_
DC = clf.dual_coef_
gamma = clf.gamma

SV = torch.from_numpy(SV)
IC = torch.tensor(IC)
DC = torch.from_numpy(DC)

# Vectorized implementation
X_expanded = X.unsqueeze(1)
SV_broadcasted = torch.broadcast_to(SV, (X.shape[0], SV.shape[0], SV.shape[1]))

res_batch = torch.subtract(SV_broadcasted, X_expanded)
res_batch = torch.norm(res_batch, p=2, dim=2)
res_batch = torch.square(res_batch)
res_batch = torch.multiply(-gamma, res_batch)
res_batch = np.exp(res_batch)
res_batch = torch.multiply(res_batch, DC)
res_batch = torch.sum(res_batch, dim=1)
res_batch = res_batch + IC


NameError: name 'clf' is not defined

In [ ]:
res_batch[0]

tensor(0.9223, dtype=torch.float64)

In [190]:
res_batch = torch.subtract(SV_broadcasted, X_expanded)
res_batch = torch.norm(res_batch, p=2, dim=2)

res_batch = torch.square(res_batch)
res_batch = torch.multiply(gamma, res_batch)
res_batch = np.exp(res_batch)
res_batch = torch.multiply(res_batch, DC)
res_batch = torch.sum(res_batch, dim=1)
res_batch = res_batch + IC

In [191]:
x_train_pred  = clf.decision_function(X_test[0].reshape(1, -1))

In [77]:

# Extra

# numpy 
"""
gamma = 0.075
all = SV - X_test[0]
all = np.square(np.linalg.norm(all, ord=2, axis=1))
all = -(all*gamma)
all = np.exp(all)
all = all * DC
all = np.sum(all, axis=1)
all = all + IC
"""

'\ngamma = 0.075\nall = SV - X_test[0]\nall = np.square(np.linalg.norm(all, ord=2, axis=1))\nall = -(all*gamma)\nall = np.exp(all)\nall = all * DC\nall = np.sum(all, axis=1)\nall = all + IC\n'

In [12]:
import torch.nn as nn
import torch

class RBF_SVM(nn.Module):
    def __init__(self, clf):
        super().__init__()
        self.SV = nn.Parameter(torch.from_numpy(clf.support_vectors_))
        self.IC = nn.Parameter(torch.tensor(clf.intercept_))
        self.DC = nn.Parameter(torch.from_numpy(clf.dual_coef_))
        self.gamma = nn.Parameter(torch.tensor(clf.gamma))
        
        

        self.SV.requires_grad = True
        self.IC.requires_grad = True
        self.DC.requires_grad = True
        self.gamma.requires_grad = True

    def forward(self, X):
        X_expanded = X.unsqueeze(1)
        SV_broadcasted = torch.broadcast_to(self.SV, (X.shape[0], self.SV.shape[0], self.SV.shape[1]))
        res_batch = torch.subtract(SV_broadcasted, X_expanded)
        res_batch = torch.norm(res_batch, p=2, dim=2)
        res_batch = torch.square(res_batch)
        res_batch = torch.multiply(-self.gamma, res_batch)
        res_batch = torch.exp(res_batch)
        res_batch = torch.multiply(res_batch, self.DC)
        res_batch = torch.sum(res_batch, dim=1)
        res_batch = res_batch + self.IC
        return res_batch

In [13]:

#from utils.rbf_model import RBF_SVM
import pickle
# Test RBF model class
rbf_path = "/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/RBF/RBF_CW_5120.pkl"
with open(rbf_path,'rb') as in_model:
    clf = pickle.load(in_model)
    
# Class RBF model
rbf_svm = RBF_SVM(clf)


In [14]:
X = torch.from_numpy(X_test)


In [15]:
adv_indices = np.where(y_test == 1)[0]


In [16]:
preds = rbf_svm(X)

In [27]:
MSELoss_svm = nn.MSELoss(reduction="mean")
targets = torch.ones(X.shape[0]).double()

res = MSELoss_svm(preds, targets).sum()

In [28]:
targets.type()

'torch.DoubleTensor'

In [27]:
targets

tensor([1., 1., 1.,  ..., 1., 1., 1.])